# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob
import time
import numpy as np
import scipy as sp
import scipy.signal
import scipy.io
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks, spectrogram
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from joblib import dump, load
import warnings

warnings.filterwarnings("ignore")


def BandpassFilter(signal, Wn, fs, N=3):
    """Bandpass filter the signal.
    Args:
        signal: The signal.
        Wn: The critial frequency or frequences.
        fs: The sampling frequency.
        N: The order of the filter.
    """
    b, a = butter(N, Wn, btype='bandpass', fs=fs)
    return filtfilt(b, a, signal)


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls


def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika DAT.mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def LoadTroikaRefFile(ref_fl):
    """
    Loads and extracts labels from a troika ref file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        bpm = LoadTroikaRefFile(ref_fls[0])

    Args:
        ref_fl: (str) filepath to a troika REF .mat file.

    Returns:
        numpy array for bpm labels.
    """
    data = sp.io.loadmat(ref_fl)['BPM0']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability.

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Ensure pr_errors and confidence_est have the same length
    min_length = min(len(pr_errors), len(confidence_est))
    pr_errors = pr_errors[:min_length]
    confidence_est = confidence_est[:min_length]

    # Now apply the boolean index
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))


def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []

    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confs = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)

    # Compute aggregate error metric
    try:
        errs = np.hstack(errs)
        confs = np.hstack(confs)
    except ValueError:
        # Ensure all arrays have the same size along dimension 0
        min_size = min(arr.shape[0] for arr in errs)
        errs = [arr[:min_size] for arr in errs]

        # Now concatenate the arrays
        errs = np.hstack(errs)
        confs = np.hstack(confs)

    return AggregateErrorMetric(errs, confs)


def CreateSpectogram(signal, fs=125, nfft=1000):
    """
    Create spectrogram from signal.

    Args:
        signal: Input signal (1D numpy array)
        fs: Sampling frequency (Hz)
        nfft: Number of FFT bins

    Returns:
        freqs: Frequency array (Hz)
        times: Time array (s)
        power: Power spectral density (1D array)
        spec: Spectrogram (2D array of power values)
    """
    # Calculate spectrogram
    freqs, times, spec = spectrogram(
        signal,
        fs=fs,
        window='triang',
        nfft=nfft,
        detrend='constant',
        scaling='density',
        mode='magnitude'
    )

    # Average power over time dimension
    power = np.mean(spec, axis=1)

    return freqs, times, power, spec


def FindDominantFrequency(freqs, power, min_freq=60, max_freq=240):
    """
    Find dominant frequency in power spectrum within physiological range
    Args:
        freqs: Frequency array (Hz)
        power: Power spectral density (1D or 2D array)
        min_freq: Minimum valid frequency (Hz)
        max_freq: Maximum valid frequency (Hz)
    Returns:
        dominant_freq: (float|None) Dominant frequency (Hz) or None if no valid peak
    """
    # Input validation
    if freqs is None or power is None:
        return None

    # Filter frequency range
    valid_idx = np.logical_and(freqs >= min_freq, freqs <= max_freq)
    valid_freqs = freqs[valid_idx]
    valid_power = power[valid_idx]

    if len(valid_power) == 0:
        return None

    # Find peaks
    peaks, _ = sp.signal.find_peaks(valid_power)

    if len(peaks) == 0:
        return None

    # Sort peaks by power and return strongest
    peak_powers = valid_power[peaks]
    sorted_idx = np.argsort(peak_powers)[::-1]
    return valid_freqs[peaks[sorted_idx[0]]]


def EstimatePulseRate(ppg_signal, acc_magnitude, feature_list, fs, Wn, peak_detect_threshold=0.5,
                      ppg_quality_threshold=0.5):
    """
    Estimate pulse rate from PPG and accelerometer signals using a machine learning model.
    Args:
        ppg_signal: Preprocessed PPG signal (1D numpy array)
        acc_magnitude: Preprocessed accelerometer magnitude signal (1D numpy array)
        feature_list: Features of the window
        fs: Sampling frequency
        Wn: Critical frequencies for bandpass filter
        peak_detect_threshold: Minimum distance between peaks (s)
        ppg_quality_threshold: Minimum signal quality index for PPG
    Returns:
        Estimated pulse rate in BPM
    """
    # Bandpass filter signals
    ppg_signal_filtered = BandpassFilter(ppg_signal, Wn, fs)
    acc_magnitude_filtered = BandpassFilter(acc_magnitude, Wn, fs)

    # Detect peaks in PPG signal
    ppg_peaks, _ = find_peaks(ppg_signal_filtered, distance=fs * peak_detect_threshold)

    # Compute pulse-to-pulse intervals
    ppg_intervals = np.diff(ppg_peaks) / fs

    # Derive heart rate from pulse-to-pulse intervals
    hr_time = 60 / np.mean(ppg_intervals) if len(ppg_intervals) > 0 else 0

    # Compute spectrogram for PPG signal
    ppg_freqs, _, ppg_power, _ = CreateSpectogram(ppg_signal_filtered, fs)

    # Compute spectrogram for accelerometer magnitude signal
    _, _, acc_magnitude_power, _ = CreateSpectogram(acc_magnitude_filtered, fs)

    # Find dominant frequency in PPG spectrum
    ppg_freq = FindDominantFrequency(ppg_freqs, ppg_power)

    # Adjust PPG frequency estimate for movement using accelerometer magnitude
    ppg_freq = AdjustPPGEstimateForMovement(acc_magnitude_power, ppg_freq, ppg_freqs, ppg_power)

    # Compute a signal quality index for the PPG
    ppg_quality = np.sum(ppg_power[(ppg_freqs >= Wn[0]) & (ppg_freqs <= Wn[1])]) / np.sum(ppg_power)

    # Use the machine learning model to predict the pulse rate
    est_bpm_ml = model.predict([feature_list])[0]

    # If motion is low and time-domain & frequency-domain agree, accept that as HR
    if ppg_quality > ppg_quality_threshold and hr_time is not None and ppg_freq is not None:
        return (hr_time + est_bpm_ml) / 2

    return est_bpm_ml


def AdjustPPGEstimateForMovement(acc_magnitude_power, ppg_freq, ppg_freqs, ppg_power):
    """
    Detect movement in accelerometer magnitude signal and adjust PPG frequency estimate.
    Args:
        acc_magnitude_power: Power spectral density of accelerometer magnitude signal.
        ppg_freq: Dominant frequency in PPG spectrum.
        ppg_freqs: Frequency array (Hz).
        ppg_power: Power spectral density (1D array).

    Returns:
        Adjusted PPG frequency estimate (float) or None if no valid peak.
    """
    # Find dominant frequency in accelerometer magnitude spectrum
    acc_freq = FindDominantFrequency(ppg_freqs, acc_magnitude_power)
    if acc_freq is not None and ppg_freq is not None and abs(acc_freq - ppg_freq) < 0.1:
        # If they are close, find the next strongest PPG frequency.
        # We do this by zeroing out the power at the current PPG frequency and finding the next strongest peak.
        ppg_power[ppg_freqs == ppg_freq] = 0
        ppg_freq = FindDominantFrequency(ppg_freqs, ppg_power)
    return ppg_freq


def PlotSpectogram(freqs, times, power, spec, title):
    """
    Plot spectrogram
    Args:
        freqs: Frequency array (Hz)
        times: Time array (s)
        power: Power spectral density (1D array)
        spec: Spectrogram (2D array of power values)
        title: Plot title
    """
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(freqs, power, color='black')
    plt.title(f"{title} Power Spectrum")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power")
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.axis(ymin=58, ymax=62)
    plt.imshow(spec, aspect='auto', cmap='magma',
               extent=(times[0], times[-1], freqs[0], freqs[-1]))
    plt.colorbar(label="Power")
    plt.title(f"{title} Spectrogram")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")
    plt.tight_layout()
    plt.show()


def ComputeConfidence(ppg, est_bpm, fs, Wn):
    """
    Compute confidence of pulse rate estimate.
    Args:
        ppg: PPG signal (1D numpy array)
        est_bpm: Estimated pulse rate in BPM
        fs: Sampling frequency
        Wn: Critical frequencies for bandpass filter
    Returns:
        confidence: Confidence value
    """
    # Bandpass filter ppg signal
    ppg = BandpassFilter(ppg, Wn, fs)

    # Compute spectrogram for PPG signal
    ppg_freqs, _, ppg_power, _ = CreateSpectogram(ppg, fs)

    # Compute the confidence by:
    # 1. Create a boolean mask for the frequency values that are within 1 Hz of the pulse rate estimate.
    # 2. Use the mask to select the spectral power values that are close to the pulse rate estimate.
    # 3. Sum the spectral power values from step 2.
    # 4. Divide the sum by the total sum of all spectral power values.
    # This shows the proportion of the total power that is concentrated close to the estimated pulse rate.
    confidence = np.sum(ppg_power[np.abs(ppg_freqs - est_bpm) <= 1]) / np.sum(ppg_power)

    return confidence


def LowpassFilter(signal, fs):
    """
    Low-pass filter the signal.
    Args:
        signal: The signal.
        param fs: The sampling frequency.
    """
    b, a = sp.signal.butter(3, 12, btype='lowpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)


def Featurize(accx, accy, accz, fs):
    """A partial featurization of the accelerometer signal.

    Args:
        accx: (np.array) x-channel of the accelerometer.
        accy: (np.array) y-channel of the accelerometer.
        accz: (np.array) z-channel of the accelerometer.
        fs: (number) the sampling rate of the accelerometer

    Returns:
        n-tuple of accelerometer features
    """

    accx = LowpassFilter(accx, fs)
    accy = LowpassFilter(accy, fs)
    accz = LowpassFilter(accz, fs)

    # The mean of the x-channel
    mn_x = np.mean(accx)

    # The standard deviation of the x-channel
    std_x = np.std(accx)

    # The 5th percentile of the x-channel
    p5_x = np.percentile(accx, 5)

    # The pearson correlation coefficient between the x and y channels
    corr_xy = sp.stats.pearsonr(accx, accy)[0]

    # The total AC energy of the x-axis
    energy_x = np.sum(np.square(accx - np.mean(accx)))

    # Take an FFT of the signal. If the signal is too short, 0-pad it so we have at least 2046 points in the FFT.
    fft_len = max(len(accx), 2046)

    # Create an array of frequency bins
    fft_freqs = np.fft.rfftfreq(fft_len, 1 / fs)

    # Take an FFT of the centered signal
    fft_x = np.fft.rfft(accx - np.mean(accx), fft_len)

    # The frequency with the most power between 0.25 and 12 Hz
    dominant_frequency_x = fft_freqs[np.argmax(np.abs(fft_x[(fft_freqs >= 0.25) & (fft_freqs <= 12)]))]

    # The fraction of energy between 2 and 3 Hz in the x-channel
    spectral_energy_x = np.square(np.abs(fft_x))
    energy_23_x = np.sum(spectral_energy_x[(fft_freqs >= 2) & (fft_freqs <= 3)]) / np.sum(spectral_energy_x)

    return (mn_x,
            std_x,
            p5_x,
            corr_xy,
            energy_x,
            dominant_frequency_x,
            energy_23_x)


def RunPulseRateAlgorithm(data_fl, ref_fl, fs=125, window_length_s=8, window_shift_s=2, Wn=(40 / 60, 240 / 60),
                          peak_detect_threshold=0.5, ppg_quality_threshold=0.5, train=False):
    """
    The algorithm for estimating pulse rate.

    This algorithm works by estimating the pulse rate from PPG and accelerometer signals using a machine learning model.
    The algorithm begins by loading the data from the data file and the reference data from the reference file. It then
    filters the signals using a bandpass filter. The accelerometer signals are combined into a single magnitude signal.

    To estimate the pulse rate, the algorithm first bandpass filters the signals and detects peaks in the PPG signal.
    It computes pulse-to-pulse intervals and derives the heart rate from these intervals. The algorithm then computes
    the spectrogram for both the PPG signal and the accelerometer magnitude signal.

    Next, it finds the dominant frequency in the PPG spectrum and adjusts the PPG frequency estimate for movement using
    the accelerometer magnitude. A signal quality index for the PPG is computed. The algorithm extracts features from
    the accelerometer signals and uses a trained machine learning model to predict the pulse rate based on these
    features. The model is selected and tuned using GridSearchCV to find the optimal hyperparameters. If the motion is
    low and the time-domain and frequency-domain estimates agree, the algorithm accepts this as the heart rate.

    Args:
        data_fl: The data file.
        ref_fl: The reference file.
        fs: Sampling frequency.
        window_length_s: Window length in seconds.
        window_shift_s: Window shift in seconds.
        Wn: Critical frequencies for bandpass filter.
        peak_detect_threshold: Minimum distance between peaks (s).
        ppg_quality_threshold: Minimum signal quality index for PPG.
        train: Whether to train the model.
    Returns:
    """
    # Load data
    global plotted, debug, predictions, features, model
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)

    # Load model if needed
    if model is None and not train:
        model = load("model.joblib")

    # Load reference data
    bpm = LoadTroikaRefFile(ref_fl)

    # Compute pulse rate estimates and estimation confidence.
    window_length = window_length_s * fs
    window_shift = window_shift_s * fs
    errors, confidence = [], []

    for i in range(0, len(ppg) - window_length, window_shift):
        # Verify index is within bpm array bounds
        bpm_idx = i // (fs * window_shift)
        if bpm_idx >= len(bpm):
            break

        ppg_window = ppg[i:i + fs * window_length]
        accx_window = accx[i:i + fs * window_length]
        accy_window = accy[i:i + fs * window_length]
        accz_window = accz[i:i + fs * window_length]

        # Combine accelerometer signals into a single magnitude signal
        # This is done by calculating the Euclidean norm of the 3D acceleration vector
        # https://en.wikipedia.org/wiki/Euclidean_space#Euclidean_norm
        acc_magnitude = np.sqrt(accx_window ** 2 + accy_window ** 2 + accz_window ** 2)

        # Get features of window
        feats = Featurize(accx_window, accy_window, accz_window, fs)
        features.append(feats)

        # Get label
        label = bpm[bpm_idx]

        # Append label
        labels.append(label)

        if not train:
            # Estimate pulse rate
            est_bpm = EstimatePulseRate(ppg_window, acc_magnitude, feats, fs, Wn, peak_detect_threshold,
                                        ppg_quality_threshold)

            # Append prediction
            predictions.append(est_bpm)

            # Calculate error
            errors.append(np.abs(est_bpm - label))

            # Compute confidence
            confidence.append(ComputeConfidence(ppg_window, est_bpm, fs, Wn))

    # Ensure errors and confidence arrays have the same length
    min_length = min(len(errors), len(confidence))
    errors = errors[:min_length]
    confidence = confidence[:min_length]

    if debug and not plotted:
        # Filter all signals
        ppg = BandpassFilter(ppg, Wn, fs)
        accx = BandpassFilter(accx, Wn, fs)
        accy = BandpassFilter(accy, Wn, fs)
        accz = BandpassFilter(accz, Wn, fs)

        # Plot spectrograms of entire signals
        ppg_freqs, ppg_times, ppg_power, ppg_spec = CreateSpectogram(ppg, fs)
        accx_freqs, accx_times, accx_power, accx_spec = CreateSpectogram(accx, fs)
        accy_freqs, accy_times, accy_power, accy_spec = CreateSpectogram(accy, fs)
        accz_freqs, accz_times, accz_power, accz_spec = CreateSpectogram(accz, fs)

        PlotSpectogram(ppg_freqs, ppg_times, ppg_power, ppg_spec, "PPG")
        PlotSpectogram(accx_freqs, accx_times, accx_power, accx_spec, "AccX")
        PlotSpectogram(accy_freqs, accy_times, accy_power, accy_spec, "AccY")
        PlotSpectogram(accz_freqs, accz_times, accz_power, accz_spec, "AccZ")
        plotted = True

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    errors, confidence = np.array(errors), np.array(confidence)
    return errors, confidence.reshape(errors.shape)


def GetAndTrainModel():
    """
    Get and train the model.
    """
    global features, labels, model

    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()

    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        RunPulseRateAlgorithm(data_fl, ref_fl, train=True)

    # Define the model
    model = RandomForestRegressor()

    # Define the parameter grid
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Perform Grid Search to find the best model
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error',
                               n_jobs=-1)
    np_features, np_labels = np.array(features), np.array(labels).flatten()
    grid_search.fit(np_features, np_labels)

    # Get the best model
    model = grid_search.best_estimator_

    # Get the cross-validation results
    cv_results = grid_search.cv_results_

    # Extract and display performance metrics
    mean_test_scores = cv_results['mean_test_score']
    std_test_scores = cv_results['std_test_score']
    params = cv_results['params']

    for mean, std, param in zip(mean_test_scores, std_test_scores, params):
        print(f"Mean: {mean}, Std: {std}, Params: {param}")

    # Save the model
    dump(model, 'model.joblib')


def InitGlobals():
    """
    Initialize global variables.
    """
    global debug, plotted, predictions, labels, features

    debug = False
    plotted = False
    predictions = []
    labels = []
    features = []


debug = False
plotted = False
predictions = []
labels = []
features = []
model = None